#### GF_0MAGN_barb

In [ ]:
%run common/gaussian_denoising.ipynb

In [ ]:
Args = namedtuple("args", "input")
args = Args("http://www.hpca.ual.es/~vruiz/images/barb.png")

In [ ]:
image = skimage_io.imread(args.input)  # Ground Truth
X = image

In [ ]:
Y = X + np.random.normal(loc=0, scale=STD_DEV, size=X.shape).reshape(X.shape)

In [ ]:
denoiser = denoising.Monochrome_Denoising(logger)

In [ ]:
def get_gaussian_kernel(sigma=1):
    number_of_coeffs = 3
    number_of_zeros = 0
    while number_of_zeros < 2 :
        delta = np.zeros(number_of_coeffs)
        delta[delta.size//2] = 1
        coeffs = scipy.ndimage.gaussian_filter1d(delta, sigma=sigma)
        number_of_zeros = coeffs.size - np.count_nonzero(coeffs)
        number_of_coeffs += 1
    return coeffs[1:-1]

In [ ]:
sigma_kernel = 4.3
sigma = np.array([sigma_kernel, sigma_kernel])
kernel = [None]*2
kernel[0] = get_gaussian_kernel(sigma[0])
kernel[1] = get_gaussian_kernel(sigma[1])
print(np.sum(kernel[0]))
plt.plot(kernel[0])
plt.show()

In [ ]:
logger.setLevel(logging.INFO)
denoised = denoiser.filter(Y, kernel)

In [ ]:
Y.shape

In [ ]:
PSNR = IT.distortion.PSNR(denoised, X)

In [ ]:
#plt.title(r"$\mathrm{GF}_{\text{" + str(sigma_kernel) + r"}}(\mathrm{Barb} + \mathbf{N}_{\mathcal{N}}), \mathbf{N}\sim \mathcal{N}_{}" + rf"(\mu=0, \sigma={std_dev})," + r"\mathrm{PSNR}=" + f"{PSNR:.2f}" + r"~\mathrm{dB}$")
string  = r"$"
string += r"\mathrm{GF}"
string += r"_\text{"
string += str(sigma_kernel)
string += r"}"
string += r"(\text{Barb} + \mathbf{N}_{\mathcal{N}(\sigma="
string += str(STD_DEV)
string += r")})"
string += r", \mathrm{PSNR}="
string += f"{PSNR:.2f}"
string += r"~\mathrm{dB}"
string += r"$"
plt.title(string)
plt.imshow(denoised, cmap="gray")
plt.savefig('GF_0MAGN_barb.pdf', bbox_inches='tight')

In [ ]:
plt.imshow(fsc.log_abs(fsc.ft2(denoised)))

#### GF_PCC_0MAGN_barb
Shows the impact of the length of the filter (sigma_kernel) for different noise levels (std_dev).

In [ ]:
sigmas_kernel = [0.25, 0.35, 0.5, 0.65, 0.75, 0.9, 1.0, 1.5]
sigmas_kernel = [0.5 + i/5 for i in range(30)] # 30
std_devs = [i for i in range(0, 120, 20)]
PCC_curves = []
#sigma_index = 1
#for std_dev in range(10, 50, int((50-10)/len(sigmas_kernel))):
for std_dev in std_devs:
    Y = X + np.random.normal(loc=0, scale=std_dev, size=X.shape).reshape(X.shape)
    #Y = np.clip(a = Y, a_min=0, a_max=255)
    PCC_curve = []
    for sigma_kernel in sigmas_kernel: #range(5, 20, 1):
        #sigma_kernel /= 10
        sigma = np.array([sigma_kernel, sigma_kernel])
        kernel = [None]*2
        kernel[0] = get_gaussian_kernel(sigma[0])
        kernel[1] = get_gaussian_kernel(sigma[1])
        denoised = denoiser.filter(Y, kernel)
        #PSNR = IT.distortion.PSNR(denoised, X)
        PCC = np.corrcoef(denoised.flatten(), X.flatten())[0, 1]
        print(std_dev, sigma_kernel, PCC)
        PCC_curve.append(PCC)
        #plt.imshow(denoised, cmap="gray")
        #plt.show()
        #input()

    PCC_curves.append(PCC_curve)
    #sigma_index += 1
    print()

In [ ]:
plt.title(r'$\mathbf{X}=\mathrm{Barb}$')
#plt.title(r"$\mathbb{E}_{\text{" + str(iters) + r"}}(\mathbf{N}" + rf"/{_lambda}" + r"), \mathbf{N}\sim\mathrm{Poisson}" + rf"(\lambda={_lambda}" + r"\mathrm{Barb})$")
#plt.title("Averaging Poisson noisy instances of \"Barb\"")
for i in range(len(PCC_curves)):
    #plt.plot([j/10 for j in range(5, 20, 1)], curves[i], label=rf"$\sigma={10+i*5}$")
    plt.plot(sigmas_kernel,
             PCC_curves[i],
             label=rf"$\sigma={std_devs[i]}$"
             + r"$, \tau^*="
             + rf"{sigmas_kernel[np.argmax(PCC_curves[i])]}$")
#plt.ylabel(r"$\mathrm{PCC}(\mathbf{X}, \hat{\mathbf{X}})$")
plt.ylabel(r"$\mathrm{PCC}(\mathbf{X}, \mathrm{GF}_\tau(\mathbf{X}+\mathbf{N}_{\mathcal{N}(\sigma)}))$")
#plt.xlabel(r"$\tau~(\text{Standard deviation of the Gaussian kernel})$")
plt.xlabel(r"$\tau$")
plt.legend()
#plt.show()
plt.savefig("GF_PCC_0MMPG_barb.pdf", bbox_inches='tight')

#### GF_SFRC_0MAGN_barb

In [ ]:
STD_DEV = 80

In [ ]:
SFRC_curve = []
sigmas_kernel = [0.25, 0.5, 0.75, 1.0, 1.2, 1.5, 1.6, 2.0, 2.5]
sigmas_kernel = [1.0, 1.25, 1.5, 1.75, 2.0, 2.25, 2.50, 2.75, 5]
#sigmas_kernel = [0.01, 0.5, 1.0, 1.25, 1.3, 1.35]
Y = X + np.random.normal(loc=0, scale=STD_DEV, size=X.shape).reshape(X.shape) # Mover fuera!!!!
#Y = np.clip(a = Y, a_min=0, a_max=255)
for i in sigmas_kernel:
    total_c_avg = 0
    kernel = [None]*2
    kernel[0] = get_gaussian_kernel(i)
    kernel[1] = get_gaussian_kernel(i)
    #total_c_avg = np.zeros(X.shape[0]>>1)
    #for j in range(1):
    denoised = denoiser.filter(Y, kernel)
    freq, c_avg = fsc.get_SFRC_curve__chessboard(denoised)
    #c_avg = fsc.single_image_frc(denoised, X.shape[0]/2)
    #freq = fsc.get_radial_spatial_frequencies(denoised, 1)
    #freq = freq[:(len(freq)>>1)]
    #c_avg = c_avg[:(len(c_avg)>>1)]
    #freq, c_avg = fsc.get_SFRC_curve(X)
    print(len(c_avg))
    #total_c_avg += c_avg
    #SFRC_curve.append(total_c_avg/1)
    SFRC_curve.append(c_avg)
    print(i, end=' ')

# Ideal curve
#freq, c_avg = fsc.get_SFRC_curve(X)
#freq = freq[:(len(freq)>>1)]
#c_avg = c_avg[:(len(c_avg)>>1)]
#SFRC_curve.append(c_avg)
#sigmas_kernel.append("orig")

In [ ]:
string  = r"$\mathbf{X}=\mathrm{Barb}$"
plt.title(string)
#plt.xlabel("Normalized spatial frequency")
plt.xlabel(r"$\omega$")
#plt.ylabel(r"$\mathrm{SFRC}(\tilde{\mathbf{X}})$")
plt.ylabel(r"$\mathrm{SFRC}(\mathrm{GF}_\tau(\mathbf{X}+\mathbf{N}_{\mathcal{N}(\sigma=" + rf"{STD_DEV}" + ")}))$")
markers = ['o', 's', '^', 'd', 'x', '*', 'v', 'p', 'h', '+']  # Add more markers if needed
#for i in range(len(sigmas_kernel)-1, 0, -1):
for i in range(len(sigmas_kernel)):
    plt.plot(freq[:(len(freq)>>0)],
             SFRC_curve[i][:(len(freq)>>0)],
             label=r"$\tau=" + f"{sigmas_kernel[i]}"
             + rf", {np.sum(SFRC_curve[i][:(len(freq)>>1)]):.2f}" # [:(len(freq)>>1)]
             + rf", {np.min(SFRC_curve[i]):.2f}"
             #+ rf", {(np.sum(SFRC_curve[i][:(len(freq)>>1)]) + np.sum(SFRC_curve[i][(len(freq)>>1):]):.2f)}"
             #+ rf", {np.sum(SFRC_curve[i]):.2f}"
             + r"$",
             marker=markers[i % len(markers)],
             markersize=0.0)
Y = X + np.random.normal(loc=0, scale=STD_DEV, size=X.shape).reshape(X.shape)
plt.plot(freq[:(len(freq)>>0)],
         fsc.get_SFRC_curve__chessboard(X)[1],
         label="No noise")
plt.plot(freq[:(len(freq)>>0)],
         fsc.get_SFRC_curve__chessboard(Y)[1],
         label=f"noise {STD_DEV}")
plt.legend()
plt.savefig("GF_SFRC_0MAGN_barb.pdf", bbox_inches='tight')

Heurística: aplicamos un tau grande (5, por ejemplo) y miramos el valor mínimo que toma la curva SFRC. Luego empezamos a usar tau's de pequeñas a grandes, y cuando alcanzamos un valor mínimo próximo a obtenido por tau grande, ese es el tau buscado.

In [ ]:
min_sigma_kernel = 0.5
max_sigma_kernel = 2.0
area_per_sigma_kernel_curves = []
std_devs = []
for s in range(20, 100, 20):
    std_devs.append(s)
    sigmas_kernel = []
    area_per_sigma_kernel_curve = []
    Y = X + np.random.normal(loc=0, scale=s, size=X.shape).reshape(X.shape) # Mover fuera!!!!
    for i in range(10):
        sigma_kernel = min_sigma_kernel + i*(max_sigma_kernel - min_sigma_kernel)/10
        sigmas_kernel.append(sigma_kernel)
        kernel = [None]*2
        kernel[0] = get_gaussian_kernel(sigma_kernel)
        kernel[1] = get_gaussian_kernel(sigma_kernel)
        denoised = denoiser.filter(Y, kernel)
        freqs, c_avg = fsc.get_SFRC_curve__chessboard(denoised)
        area = np.sum(c_avg)
        area_per_sigma_kernel_curve.append(area)
    area_per_sigma_kernel_curves.append(area_per_sigma_kernel_curve)

In [ ]:
for i in range(len(area_per_sigma_kernel_curves)):
    plt.plot(
        sigmas_kernel,
        area_per_sigma_kernel_curves[i],
    label=f"{std_devs[i]}")
plt.legend()

En cuanto rebota, dividimos entre 2 tau y esa es la estimación para la longitud del filtro Gaussiano. Dicha estimación no depende del nivel de ruido (que es deconocido), sino de la imagen ruidosa en su conjunto.